## National Stock Exchange of India for all series. 

A brief description of columns:
* SYMBOL: Symbol of the listed company. 
* SERIES: Series of the equity. Values are [EQ, BE, BL, BT, GC and IL] 
* OPEN: The opening market price of the equity symbol on the date. 
* HIGH: The highest market price of the equity symbol on the date. 
* LOW: The lowest recorded market price of the equity symbol on the date. 
* CLOSE: The closing recorded price of the equity symbol on the date. 
* LAST: The last traded price of the equity symbol on the date. 
* PREVCLOSE: The previous day closing price of the equity symbol on the date. 
* TOTTRDQTY: Total traded quantity of the equity symbol on the date. 
* TOTTRDVAL: Total traded volume of the equity symbol on the date. 
* TIMESTAMP: Date of record. 
* TOTALTRADES: Total trades executed on the day. 
* ISIN: International Securities Identification Number.

In [2]:
spark

In [3]:
%fs ls /FileStore/tables/

In [4]:
%fs head /FileStore/tables/FINAL_FROM_DF.csv

In [5]:
from pyspark.sql.types import *
inputPath="/FileStore/tables/"
# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
csvSchema = StructType([ StructField("SYMBOL", StringType(), True), StructField("SERIES", StringType(), True),StructField("OPEN", FloatType(), True),StructField("HIGH", FloatType(), True),StructField("LOW", FloatType(), True),StructField("CLOSE", FloatType(), True),StructField("LAST", FloatType(), True),StructField("PREVCLOSE", FloatType(), True),StructField("TOTTRDQTY", FloatType(), True),StructField("TOTTRDVAL", FloatType(), True),StructField("TIMESTAMP", TimestampType(), True),StructField("TOTALTRADES", FloatType(), True),StructField("ISIN", StringType(), True)])

# Static DataFrame representing data in the CSV file
staticInputDF = (
  spark
    .read
    .schema(csvSchema)
    .csv(inputPath)
)

display(staticInputDF)

In [6]:
from pyspark.sql.functions import *      # for window() function

staticCountsDF = (
  staticInputDF
    .groupBy(
       staticInputDF.SERIES, 
       window(staticInputDF.TIMESTAMP, "1 day"))    
    .count()
)
staticCountsDF.cache()

# Register the DataFrame as table 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")

In [7]:
%sql select SERIES,sum(count) as total_count from static_counts where SERIES IN ('EQ','NC','BZ') group by SERIES

In [8]:
%sql select SERIES, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts 
where SERIES IN ('EQ','NC','BZ') order by 'TIMESTAMP', SERIES

In [9]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(csvSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .csv(inputPath)
)

# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.SERIES, 
      window(streamingInputDF.TIMESTAMP, "1 day"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

In [10]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

In [11]:
from time import sleep
sleep(5)  # wait a bit for computation to start

In [12]:
%sql select SERIES, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts 
where SERIES IN ('EQ','NC','BZ') order by 'TIMESTAMP', SERIES

In [13]:
sleep(30)  # wait a bit more for more data to be computed

In [14]:
%sql select SERIES, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts 
where SERIES IN ('EQ','NC','BZ') order by 'TIMESTAMP', SERIES

In [15]:
%sql select SERIES,sum(count) as total_count from static_counts where SERIES IN ('EQ','NC','BZ') group by SERIES